# Automation of Acquisition for Basler Camera #
- find a way to automate between the scanner (pfm450) and basler camera for faster data collection
- send command to the scanner to start scan
    - scanner for the lab is closed loop
    - PFM 450: objective scanner that uses precise voltage control to adjust the position of an optical element
        - a piezoelectric actuator : works by adding voltage to it, which would either expand or contract the element. This mechanical movement is used to adjust the position of an attached optical component, such as an objective lens.
- command camera to acquire images once the scanner reaches the appropriate position (settles at the right level)

In [2]:
import os
import time
import sys
import clr
from decimal import Decimal  
from pypylon import pylon
import cv2

# for windows only..
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.GenericPiezoCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.Benchtop.PrecisionPiezoCLI.dll")

from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.GenericPiezoCLI import Piezo
from Thorlabs.MotionControl.Benchtop.PrecisionPiezoCLI import BenchtopPrecisionPiezo

RuntimeError: Failed to create a default .NET runtime, which would
                    have been "mono" on this system. Either install a
                    compatible runtime or configure it explicitly via
                    `set_runtime` or the `PYTHONNET_*` environment variables
                    (see set_runtime_from_env).

In [ ]:
# send command to scanner to start scanning
def initialize_scanner(serial_no):
    """
    Initialize the PFM450 scanner in closed-loop mode.
    * Code obtained from Thorlabs Motion Control github repository - uses DLLs but NOT supported on Macs.
    """
    DeviceManagerCLI.BuildDeviceList()

    # Connect, begin polling, and enable
    device = BenchtopPrecisionPiezo.CreateBenchtopPiezo(serial_no)
    device.Connect(serial_no)
    
    # Ensure that the device settings have been initialized
    channel = device.GetChannel(1)
    
    # Wait for settings to initialize.
    if not channel.IsSettingsInitialized():
        channel.WaitForSettingsInitialized(10000)  # 10 seconds timeout
        if not channel.IsSettingsInitialized():
            raise Exception("Device settings failed to initialize.")
    
    # Start polling and enable
    channel.StartPolling(250)   # 250 ms polling rate
    time.sleep(0.25)
    channel.EnableDevice()
    time.sleep(0.25)            # waiting for device to enable
    
    # Get Device Information and display description
    device_info = channel.GetDeviceInfo()
    print("device info: ", device_info.Description)

    # Load any configuration settings needed by the controller/stage
    piezoConfiguration = channel.GetPiezoConfiguration(serial_no)
    print("Piezo Configuration: ", piezoConfiguration)

    # Closed loop mode
    channel.SetPositionControlMode(Piezo.PiezoControlModeTypes.CloseLoop)
    time.sleep(0.25)
    
    return device, channel

In [ ]:
def move_scanner_to(channel, target_position, tolerance=Decimal("0.5")):
    """
    Commands the scanner to move to target_position (in µm) and waits until the position is reached.
    """
    max_position = channel.GetMaxTravel()
    print(f"Maximum Travel (µm): {max_position}")
    
    if target_position > max_position:
        raise Exception("Target position exceeds maximum travel range.")

    # move scanner to target position
    channel.SetPosition(target_position)
    print(f"Moving scanner to: {target_position} µm")
    
    while True:
        current_position = channel.GetPosition()
        print(f"Current Position (µm): {current_position}")
        if abs(current_position - target_position) <= tolerance:
            print("Target position reached.")
            break
        time.sleep(0.1)  # poll every 100ms

In [ ]:
def initialize_camera():
    """
    Initialize the Basler camera using the pypylon library.
    """
    camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
    camera.Open()

    # demonstrate some feature access
    new_width = camera.Width.Value - camera.Width.Inc
    if new_width >= camera.Width.Min:
        camera.Width.Value = new_width
    
    camera.ExposureTime.SetValue(10000)     # Set exposure to 10ms (adjust as needed)
    camera.TriggerMode.SetValue("Off")      # using software triggering for automation so this should be off
    return camera

In [ ]:
def capture_image(camera, image_filename):
    """
    Captures a single image with the Basler camera and saves it.
    """
    # Grab one image
    camera.StartGrabbingMax(1)
    
    # Retrieve the image from the camera (blocking call with a timeout)
    grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

    if grabResult.GrabSucceeded():
        # Access the image data.
        print("SizeX: ", grabResult.Width)
        print("SizeY: ", grabResult.Height)
        image = grabResult.Array
        cv2.imwrite(image_filename, image)
        print(f"Image saved to {image_filename}")
    else:
        print("Error: Failed to grab image from camera.")
    
    grabResult.Release()

In [ ]:
def main():
    scanner_serial_no = "44000001"  # serial number of the scanner
    target_positions = [Decimal("100"), Decimal("150"), Decimal("200")]
    
    try:
        device, channel = initialize_scanner(scanner_serial_no)
    except Exception as e:
        print("Error initializing scanner:", e)
        return
    
    try:
        camera = initialize_camera()
    except Exception as e:
        print("Error initializing camera:", e)
        return

    for pos in target_positions:
        try:
            print(f"Moving to position: {pos} µm")
            move_scanner_to(channel, pos)
            
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            image_filename = f"image_{pos}um_{timestamp}.png"
            
            capture_image(camera, image_filename)
            
            time.sleep(1)
        except Exception as e:
            print("Error during movement or image capture:", e)
    
    channel.StopPolling()
    channel.Disconnect()
    camera.Close()
    print("Automation complete. Scanner and camera disconnected.")

In [ ]:
# grab camera - code from github

# from pypylon import pylon

# camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
# camera.Open()

# # demonstrate some feature access
# new_width = camera.Width.Value - camera.Width.Inc
# if new_width >= camera.Width.Min:
#     camera.Width.Value = new_width

# numberOfImagesToGrab = 100
# camera.StartGrabbingMax(numberOfImagesToGrab)

# while camera.IsGrabbing():
#     grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

#     if grabResult.GrabSucceeded():
#         # Access the image data.
#         print("SizeX: ", grabResult.Width)
#         print("SizeY: ", grabResult.Height)
#         img = grabResult.Array
#         print("Gray value of first pixel: ", img[0, 0])

#     grabResult.Release()
# camera.Close()